On commence par importer les bibliothèques nécessaires. Attention : NE PAS EFFACER !

In [ ]:
from telemetrix import telemetrix     # pour piloter la carte Arduino en temps réel
from pylab import *    # charge toutes les bibliothèques scientifiques
from scipy.optimize import curve_fit    # permet de modéliser selon un modèle personnalisé
from lmfit import minimize,Parameters,Parameter,report_fit
from lmfit.models import ExpressionModel
from time import *
%matplotlib inline
rcParams['figure.figsize'] = [32, 16]    # à enlever si on utilise %matplotlib widget
rcParams['font.size'] = 20    # pour un texte plus grand
rcParams['lines.markersize'] = 15    # pour des symboles plus grands
rcParams['lines.markeredgewidth'] = 2    # pour des symboles plus larges

Initialisation des valeurs et mesure automatique des données :

In [ ]:
nbMesures = 1    # le nombre de mesures qu'on désire réaliser à chaque fois
i = nbMesures    # pour ne pas démarrer les mesures dès le départ
P = []
variable = []    # nommer la variable de façon adéquate, et remplir son tableau de valeurs au fur et à mesure.
Analog_pin = 0
# Callback data indices
CB_PIN_MODE = 0
CB_PIN = 1
CB_VALUE = 2
CB_TIME = 3
carte = telemetrix.Telemetrix("/dev/ttyACM0")    # on définit la carte Arduino sous le nom de "carte"
def MonCallback(data) :    # on définit ici ce qui sera exécuté à chaque appel de mesure
    global i, nbMesures, P
    offset = 63.5
    echelle = 1059
    raw_value = data[CB_VALUE]
    if i < nbMesures :    # pour ne réaliser que le nombre de mesures demandé
        pression = (raw_value - offset) * echelle
        P.append(pression)
        i+=1

EA0 = carte.set_pin_mode_analog_input(Analog_pin, callback = MonCallback)    # définition de la voie d'acquisition

Acquisition des valeurs.  
Entrer une nouvelle valeur de la variable pour réaliser une nouvelle mesure. Pour arrêter, valider sans entrer de valeur.

In [ ]:
while True :
    entree = input("Entrer la valeur du paramètre :")
    try :
        variable.append(float(entree))
        i = 0    # initialisation du compteur
        carte.enable_analog_reporting(Analog_pin)    # on permet la mesur des valeurs sur la voie d'entrée
        sleep(0.1)    # courte attente le temps que la mesure soit effectuée
        carte.disable_analog_reporting(Analog_pin)    # on arrête les mesures
    except :
        carte.shutdown()
        break

In [ ]:
variable = array(variable)    # Donner un nom adéquat à la nouvelle variable. Cette commande transforme une liste en tableau de valeurs exploitable
abscisse = "x"    # choix de la grandeur en abscisse, pour le graphique et pour le modèle
ordonnee = "y"    # choix de la grandeur en ordonnée, pour le graphique et pour le modèle
equation = "a * x + b"    # expression de l'équation correspondant au modèle recherché ; la variable peut être x ou le nom donné en abscisse

Calcul des caractéristiques de la courbe de régression. Attention : NE PAS SUPPRIMER, et ne le modifier que si les paramètres par défaut ne donnent pas de bon résultat :

In [ ]:
def Modele(expression, x, y):
    
    modele = ExpressionModel(expression)    # si la variable indépendante n'est pas "x", ajouter comme argument independant_vars = ["nom"])

    # Cette partie devine les paramètres et leur attribue une valeur initiale de 1 pour la recherche
    parametres = modele.make_params()
    for i in parametres :
        modele.set_param_hint(i, value = 1)
    parametres = modele.make_params()

    #modele.set_param_hint('a', value = 1, vary = True, min = -inf, max = inf)    # À compléter pour changer les valeurs initiales des paramètres. Le dernier argument est expr (string)

    resultat = modele.fit(y, parametres, x = x)    # Entrer ici le nom de la variable en x et en y, pour le calcul final des paramètres

    # On affiche les valeurs trouvées pour les paramètres :
    valeurs = ""
    for key in resultat.params:
        valeurs += f"{key} = {resultat.params[key].value:.3g} ; incertitude : {resultat.params[key].stderr:.2g}\n"
    return(modele, resultat.params, valeurs, expression)

## On commence par arranger les expressions avec "x" comme variable, car lmfit travaille avec x par défaut :
abscisse1 = abscisse
ordonnee1 = ordonnee
equation1 = equation
equation = re.sub(r"\b"+abscisse+r"\b","x", equation)
abscisse = globals()[abscisse]
ordonnee = globals()[ordonnee]

# On calcule maintenant les grandeurs liées au modèle :
xMod = linspace(min(abscisse), max(abscisse), 30)    # Intervalle de valeurs pour le modèle : (début, fin, nombre de points)
modele, parametres, valeurs, expression = Modele(equation, abscisse, ordonnee)    # Expression du modèle, et variables en abscisse et en ordonnée
expression = f"{ordonnee1} = {equation1}"
yMod = modele.eval(parametres, x = xMod)

# Les paramètres sont définis de façon globale, et utilisables dans des calculs :
for key in parametres:
    globals()[key] = parametres[key].value

Tracé de graphique(s) à partir des tableaux précédents :

In [ ]:
subplot(1, 1, 1)    # lignes, colonnes, numéro du graphique

# Tracé des points expérimentaux :
plot(abscisse, ordonnee, '+', label="")    # abscisse, ordonnée, options : + o . , s points   - -- -. : lignes  b g r y m c k w couleurs ; label est la légende associée à la courbe

# Tracé du modèle :
plot(xMod, yMod, '-', label=f"Modèle :    {expression} \n {valeurs}" )    # trace la courbe du modèle

#axis('equal')    # à décommenter si on veut un repère orthonormé
grid()    # ajoute une grille
legend()    # ajoute la légende au graphique
title("Mon titre")    # mettre le titre désiré entre les guillemets
xlabel("")    # mettre le nom de l'axe des abscisses entre les guillemets
ylabel("");    # mettre le nom de l'axe des ordonnées entre les guillemets. Le point-virgule final évite d'afficher un texte récapitulatif.

Calcul de valeurs particulières :